In [1]:
# example of a cnn for image classification
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import (Conv2D, DepthwiseConv2D,
                                     SeparableConv2D, AveragePooling2D,
                                     Flatten, InputLayer, ZeroPadding2D)
from tqdm.notebook import tqdm

In [2]:
from benchmark_utils import run_benchmarks
from parse_utils import parse_benchmark_results
from utils import abs_listdir


N_CLASSES = 1000
IN_SHAPE = (224, 224, 3)


def create_conv(out_chan, kernel, padding=None, **params):
    # TODO: take a look at SeparableConv2D, you haven't used it!
    block = []
    if padding:
        block += [ZeroPadding2D(padding=padding)]
    activation = params['activation']
    if params.get('depthwise_separable'):
        if params.get('spatial_separable'):
            block += [
                DepthwiseConv2D(kernel_size=(kernel, 1)),
                DepthwiseConv2D(kernel_size=(1, kernel)),
            ]
        else:
            block += [
                DepthwiseConv2D(kernel_size=kernel),
            ]
        block += [
            Conv2D(filters=out_chan, kernel_size=1, activation=activation)
        ]
    else:
        if params.get('spatial_separable'):
            block += [ # TODO: How to do this correctly?
                Conv2D(out_chan, kernel_size=(kernel, 1)),
                DepthwiseConv2D(kernel_size=(1, kernel), activation=activation),
            ]
        else:
            block += [
                Conv2D(out_chan, kernel_size=kernel, activation=activation)
            ]
    return block


def create_model(**params):
    model = Sequential([
        InputLayer(input_shape=IN_SHAPE),
        *create_conv(out_chan=6, kernel=5, **params),
        AveragePooling2D(pool_size=(2,2)),
        *create_conv(out_chan=16, kernel=5, **params),
        AveragePooling2D(pool_size=(2,2)),
        *create_conv(out_chan=120, kernel=5, **params),
        Flatten(),
        # NOTE: sometimes we can use different activations
        Dense(84, activation=params['activation']),
        Dense(N_CLASSES, activation='softmax')
    ])

    return model


def create_model_mobilenet(**params):
    model = Sequential([
        InputLayer(input_shape=IN_SHAPE),
        *create_conv(out_chan=32, kernel=3, padding=[1, 1], **params),
        AveragePooling2D(pool_size=(2, 2)),

        *create_conv(out_chan=64, kernel=3, padding=[1, 1], **params),
        AveragePooling2D(pool_size=(2, 2)),
        
        *create_conv(out_chan=128, kernel=3, padding=[1, 1], **params),
        AveragePooling2D(pool_size=(2, 2)),
        
        *create_conv(out_chan=256, kernel=3, padding=[1, 1], **params),
        AveragePooling2D(pool_size=(2, 2)),
        
        *create_conv(out_chan=512, kernel=3, padding=[1, 1], **params),
        AveragePooling2D(pool_size=(2, 2)),

        *create_conv(out_chan=1024, kernel=3, padding=[1, 1], **params),
        AveragePooling2D(pool_size=(7, 7)),
        # 1x1x1024

        Flatten(),
        Dense(N_CLASSES, activation='softmax')
    ])
    return model


def generate_models(params_list, model_creator):
    for params in params_list:
        name = params['activation'] + ('_depthwise' if params['depthwise_separable'] else '') + ('_spatial' if params['spatial_separable'] else '') 
        model = model_creator(**params)
        yield name, model

In [3]:
# define model
models_params = []
for depth in [True, False]:
    for spat in [True, False]:
        for act in ['tanh', 'relu', 'elu']:
            models_params.append({'activation':act, 'depthwise_separable': depth, 'spatial_separable': spat})


# printing model summaries
for name, model in generate_models(models_params, create_model_mobilenet):
    print('-'*80)
    print('NAME ' + name)
    model.summary()
    del model

--------------------------------------------------------------------------------
NAME tanh_depthwise_spatial
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 224, 226, 3)       12        
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 32)      128       
_________________________________________________________________
average_pooling2d (AveragePo (None, 112, 112, 32)      0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 114, 114, 32)      0         
_____________

## Converting models

In [7]:
# Converting the models to tflite format and saving them
from tqdm import tqdm
models_path = "tflite_models/separable"
os.makedirs(models_path, exist_ok=True)

for name, model in tqdm(generate_models(models_params, create_model_mobilenet)):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    # input('continue?')
    tflite_model = converter.convert()

    # write it to file
    filepath = f'{models_path}/{name}.tflite'
    with open(filepath, 'wb') as f:
        f.write(tflite_model)
    del tflite_model

12it [00:43,  3.60s/it]


**Benchmark tool:** https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/tools/benchmark

In [8]:
output_dir = 'results/separable'
!python profiler.py --models-dir {models_path} --output-dir {output_dir}

0%|                                                    | 0/12 [00:00<?, ?it/s]Running: /home/loveml/Downloads/tensorflow/bazel-bin/tensorflow/lite/tools/benchmark/benchmark_model \
--graph=/home/loveml/Desktop/dsproject/tflite_models/separable/elu.tflite \
--enable_op_profiling=true \
--profiling_output_csv_file=/home/loveml/Desktop/dsproject/results/separable/elu
STARTING!
Duplicate flags: num_threads
Min num runs: [50]
Min runs duration (seconds): [1]
Max runs duration (seconds): [150]
Inter-run delay (seconds): [-1]
Num threads: [1]
Benchmark name: []
Output prefix: []
Min warmup runs: [1]
Min warmup runs duration (seconds): [0.5]
Graph: [/home/loveml/Desktop/dsproject/tflite_models/separable/elu.tflite]
Input layers: []
Input shapes: []
Input value ranges: []
Input layer values files: []
Allow fp16 : [0]
Require full delegation : [0]
Enable op profiling: [1]
Max profiling buffer entries: [1024]
CSV File to export profiling data to: [/home/loveml/Desktop/dsproject/results/separable/

In [21]:
# display one of the csv files
import pandas as pd
pd.read_csv(output_dir + '/relu_spatial.csv')

,node type,start,first,avg_ms,%,cdf%,mem KB,times called,name
0,PAD,0.00000,0.115,0.18518,0.191647%,0.191647%,0,1,[sequential_38/zero_padding2d_229/Pad]:0
1,CONV_2D,0.18542,8.583,5.70974,5.90913%,6.10078%,0,1,[sequential_38/conv2d_228/Conv2D]:1
2,DEPTHWISE_CONV_2D,5.89558,13.713,14.63890,15.1501%,21.2509%,0,1,[sequential_38/depthwise_conv2d_318/Relu]:2
3,AVERAGE_POOL_2D,20.53480,3.184,2.73116,2.82654%,24.0774%,0,1,[sequential_38/average_pooling2d_228/AvgPool]:3
4,PAD,23.26640,0.191,0.25104,0.259807%,24.3372%,0,1,[sequential_38/zero_padding2d_230/Pad]:4
5,CONV_2D,23.51780,10.168,11.56840,11.9724%,36.3097%,0,1,[sequential_38/conv2d_229/Conv2D]:5
6,DEPTHWISE_CONV_2D,35.08660,6.133,6.78522,7.02217%,43.3318%,0,1,[sequential_38/depthwise_conv2d_319/Relu]:6
7,AVERAGE_POOL_2D,41.87220,0.851,1.07344,1.11093%,44.4427%,0,1,[sequential_38/average_pooling2d_229/AvgPool]:7
8,PAD,42.94600,0.095,0.12224,0.126509%,44.5693%,0,1,[sequential_38/zero_padding2d_231/Pad]:8
9,CONV_2D,43.06850,9.233,10.76640,11.1423%,55.7116%,0,1,[sequential_38/conv2d_230/Conv2D]:9


## Quantizing models

In [5]:
# Converting the models to tflite format and saving them
from tqdm import tqdm
models_path = "tflite_models/separable_quant"
os.makedirs(models_path, exist_ok=True)
# import tensorflow.lite

for name, model in tqdm(generate_models(models_params, create_model_mobilenet)):
    name += '_quant'
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.compat.v1.lite.constants.FLOAT16]
    tflite_model = converter.convert()

    # write it to file
    filepath = f'{models_path}/{name}.tflite'
    with open(filepath, 'wb') as f:
        f.write(tflite_model)
    del tflite_model

12it [00:45,  3.76s/it]


In [6]:
output_dir = 'results/separable_quant'
!python profiler.py --models-dir {models_path} --output-dir {output_dir}

0%|                                                    | 0/12 [00:00<?, ?it/s]Running: /home/loveml/Downloads/tensorflow/bazel-bin/tensorflow/lite/tools/benchmark/benchmark_model \
--graph=/home/loveml/Desktop/dsproject/tflite_models/separable_quant/tanh_spatial_quant.tflite \
--enable_op_profiling=true \
--profiling_output_csv_file=/home/loveml/Desktop/dsproject/results/separable_quant/tanh_spatial_quant
STARTING!
Duplicate flags: num_threads
Min num runs: [50]
Min runs duration (seconds): [1]
Max runs duration (seconds): [150]
Inter-run delay (seconds): [-1]
Num threads: [1]
Benchmark name: []
Output prefix: []
Min warmup runs: [1]
Min warmup runs duration (seconds): [0.5]
Graph: [/home/loveml/Desktop/dsproject/tflite_models/separable_quant/tanh_spatial_quant.tflite]
Input layers: []
Input shapes: []
Input value ranges: []
Input layer values files: []
Allow fp16 : [0]
Require full delegation : [0]
Enable op profiling: [1]
Max profiling buffer entries: [1024]
CSV File to export profil

## Dynamic range quantization

In [9]:
# Converting the models to tflite format and saving them
from tqdm import tqdm
models_path = "tflite_models/separable_quant_dr"
os.makedirs(models_path, exist_ok=True)
# import tensorflow.lite

for name, model in tqdm(generate_models(models_params, create_model_mobilenet)):
    name += '_quant'
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()

    # write it to file
    filepath = f'{models_path}/{name}.tflite'
    with open(filepath, 'wb') as f:
        f.write(tflite_model)
    del tflite_model

output_dir = 'results/separable_quant_dr'
!python profiler.py --models-dir {models_path} --output-dir {output_dir}

0%|                                                    | 0/12 [00:00<?, ?it/s]Running: /home/loveml/Downloads/tensorflow/bazel-bin/tensorflow/lite/tools/benchmark/benchmark_model \
--graph=/home/loveml/Desktop/dsproject/tflite_models/separable_quant_dr/tanh_spatial_quant.tflite \
--enable_op_profiling=true \
--profiling_output_csv_file=/home/loveml/Desktop/dsproject/results/separable_quant_dr/tanh_spatial_quant

STARTING!
Duplicate flags: num_threads
Min num runs: [50]
Min runs duration (seconds): [1]
Max runs duration (seconds): [150]
Inter-run delay (seconds): [-1]
Num threads: [1]
Benchmark name: []
Output prefix: []
Min warmup runs: [1]
Min warmup runs duration (seconds): [0.5]
Graph: [/home/loveml/Desktop/dsproject/tflite_models/separable_quant_dr/tanh_spatial_quant.tflite]
Input layers: []
Input shapes: []
Input value ranges: []
Input layer values files: []
Allow fp16 : [0]
Require full delegation : [0]
Enable op profiling: [1]
Max profiling buffer entries: [1024]
CSV File to exp